In [1]:
import pandas as pd
import joblib

# Load model
model = joblib.load("../models/ev_demand_model.pkl")

# Load data (for reference)
df = pd.read_csv("../data/ev_charging_data_preprocessed.csv")

df.head()


,date,hour,station_id,energy_kwh,day_of_week,is_weekend,month
0,2025-01-01,0,0,10.63,2,0,1
1,2025-01-01,0,1,17.10,2,0,1
2,2025-01-01,0,2,18.01,2,0,1
3,2025-01-01,0,3,10.33,2,0,1
4,2025-01-01,0,4,14.38,2,0,1


In [2]:
# Define context
context = {
    "hour": 18,
    "station_id": 1,
    "day_of_week": 2,
    "is_weekend": 0,
    "month": 6
}

# Ensure column order matches training
feature_order = ["hour", "station_id", "day_of_week", "is_weekend", "month"]

input_df = pd.DataFrame([context])[feature_order]

predicted_demand = model.predict(input_df)[0]

print("Predicted Demand:", round(predicted_demand, 2))





Predicted Demand: 28.0


In [3]:
def decision_engine(predicted_demand, capacity=40):
    utilization = predicted_demand / capacity
    
    if utilization < 0.7:
        return "LOW RISK - Normal operation"
    elif utilization < 1.0:
        return "MEDIUM RISK - Monitor closely"
    else:
        return "HIGH RISK - Shift load or mitigate"

decision = decision_engine(predicted_demand)

print("Decision:", decision)


Decision: LOW RISK - Normal operation


In [4]:
def simulate_scenario(base_demand, growth=1.0, stress=1.0):
    return base_demand * growth * stress

# Example: 25% EV growth + 20% stress day
simulated = simulate_scenario(predicted_demand, growth=1.25, stress=1.2)

print("Simulated Future Demand:", round(simulated, 2), "kWh")
print("Decision Under Scenario:", decision_engine(simulated))


Simulated Future Demand: 42.0 kWh
Decision Under Scenario: HIGH RISK - Shift load or mitigate


In [5]:
def multi_step_forecast(model, context, steps=6, capacity=40):
    results = []
    current = context.copy()
    
    for i in range(steps):
        input_df = pd.DataFrame([current])
        pred = model.predict(input_df)[0]
        
        results.append({
            "hour": current["hour"],
            "predicted_demand": round(pred, 2),
            "decision": decision_engine(pred, capacity)
        })
        
        # move time forward
        current["hour"] = (current["hour"] + 1) % 24
        
    return pd.DataFrame(results)

forecast_df = multi_step_forecast(model, context, steps=6)

forecast_df


,hour,predicted_demand,decision
0,18,28.00,LOW RISK - Normal operation
1,19,26.99,LOW RISK - Normal operation
2,20,27.48,LOW RISK - Normal operation
3,21,27.44,LOW RISK - Normal operation
4,22,12.22,LOW RISK - Normal operation
5,23,12.37,LOW RISK - Normal operation


In [6]:
def capacity_risk_analysis(forecast_df, capacity=40):
    
    forecast_df["utilization"] = forecast_df["predicted_demand"] / capacity
    
    total_steps = len(forecast_df)
    high_risk_hours = len(forecast_df[forecast_df["utilization"] >= 1.0])
    medium_risk_hours = len(forecast_df[
        (forecast_df["utilization"] >= 0.7) &
        (forecast_df["utilization"] < 1.0)
    ])
    
    peak_utilization = forecast_df["utilization"].max()
    
    summary = {
        "total_hours_forecasted": total_steps,
        "high_risk_hours": high_risk_hours,
        "medium_risk_hours": medium_risk_hours,
        "peak_utilization_ratio": round(peak_utilization, 2),
        "capacity_upgrade_needed": high_risk_hours > 2
    }
    
    return summary


In [7]:
{
 'total_hours_forecasted': 6,
 'high_risk_hours': 2,
 'medium_risk_hours': 1,
 'peak_utilization_ratio': 1.18,
 'capacity_upgrade_needed': False
}


{'total_hours_forecasted': 6,
 'high_risk_hours': 2,
 'medium_risk_hours': 1,
 'peak_utilization_ratio': 1.18,
 'capacity_upgrade_needed': False}

In [8]:
def ev_intelligence_system(
    model,
    context,
    capacity=40,
    growth=1.0,
    stress=1.0,
    steps=6
):
    
    # Step 1: Base Prediction
    base_df = pd.DataFrame([context])
    base_prediction = model.predict(base_df)[0]
    
    # Step 2: Scenario Simulation
    simulated_base = base_prediction * growth * stress
    
    # Step 3: Multi-step Forecast
    results = []
    current_context = context.copy()
    
    for step in range(steps):
        input_df = pd.DataFrame([current_context])
        pred = model.predict(input_df)[0]
        
        # apply scenario multiplier
        pred = pred * growth * stress
        
        utilization = pred / capacity
        
        results.append({
            "hour": current_context["hour"],
            "predicted_demand": round(pred, 2),
            "utilization": round(utilization, 2)
        })
        
        current_context["hour"] = (current_context["hour"] + 1) % 24
    
    forecast_df = pd.DataFrame(results)
    
    # Step 4: Risk Summary
    high_risk_hours = len(forecast_df[forecast_df["utilization"] >= 1.0])
    peak_utilization = forecast_df["utilization"].max()
    
    summary = {
        "base_prediction": round(base_prediction, 2),
        "simulated_base_prediction": round(simulated_base, 2),
        "high_risk_hours": high_risk_hours,
        "peak_utilization_ratio": round(peak_utilization, 2),
        "capacity_upgrade_needed": high_risk_hours > 2
    }
    
    return forecast_df, summary


In [9]:
forecast_df, summary = ev_intelligence_system(
    model=model,
    context=context,
    capacity=40,
    growth=1.25,   # 25% growth
    stress=1.1,    # stress day
    steps=6
)


In [10]:
forecast_df


,hour,predicted_demand,utilization
0,18,38.50,0.96
1,19,37.11,0.93
2,20,37.78,0.94
3,21,37.73,0.94
4,22,16.81,0.42
5,23,17.01,0.43


In [11]:
summary

{'base_prediction': np.float64(28.0),
 'simulated_base_prediction': np.float64(38.5),
 'high_risk_hours': 0,
 'peak_utilization_ratio': np.float64(0.96),
 'capacity_upgrade_needed': False}

In [14]:
def advanced_insight_layer(forecast_df, capacity=40):
    
    avg_demand = forecast_df["predicted_demand"].mean()
    max_demand = forecast_df["predicted_demand"].max()
    min_demand = forecast_df["predicted_demand"].min()
    
    overload_hours = len(forecast_df[forecast_df["utilization"] >= 1.0])
    overload_percentage = (overload_hours / len(forecast_df)) * 100
    
    stress_index = forecast_df["utilization"].mean()
    
    insights = {
        "average_forecast_demand": float(round(avg_demand, 2)),
        "peak_forecast_demand": float(round(max_demand, 2)),
        "minimum_forecast_demand": float(round(min_demand, 2)),
        "overload_percentage": float(round(overload_percentage, 2)),
        "infrastructure_stress_index": float(round(stress_index, 2))
    }
    
    return insights


In [15]:
insights = advanced_insight_layer(forecast_df)

insights


{'average_forecast_demand': 30.82,
 'peak_forecast_demand': 38.5,
 'minimum_forecast_demand': 16.81,
 'overload_percentage': 0.0,
 'infrastructure_stress_index': 0.77}

In [16]:
def strategic_decision_layer(insights, capacity_threshold=0.85):
    
    stress = insights["infrastructure_stress_index"]
    overload = insights["overload_percentage"]
    
    if overload > 30:
        level = "CRITICAL"
        recommendation = "Immediate capacity expansion required"
        
    elif stress >= capacity_threshold:
        level = "WARNING"
        recommendation = "Monitor closely and prepare mitigation strategy"
        
    else:
        level = "STABLE"
        recommendation = "Infrastructure operating within safe limits"
    
    return {
        "system_status": level,
        "final_recommendation": recommendation
    }


In [17]:
final_decision = strategic_decision_layer(insights)

final_decision


{'system_status': 'STABLE',
 'final_recommendation': 'Infrastructure operating within safe limits'}